In [1]:
from selenium import webdriver

#opt.add_argument('headless')
#opt = webdriver.ChromeOptions()
#dr=webdriver.Chrome('C:/Users/wjssm/Documents/chromedriver.exe',chrome_options = opt) #nothings happen

dr = webdriver.Chrome('C:/Users/wjssm/Documents/chromedriver.exe')


In [2]:
pageNo = str(100)
dr.get('https://music.bugs.co.kr/newest/album/kpoppicked?page='+pageNo)

- 한 페이지당 7 x 10 70개의 앨범
- 대략 100~110 만 긁어보자

`고민거리`
- 새로운 노래가 업데이트 되면 페이지도 달라지는 거 아닌가..?

In [3]:
import pandas as pd
import numpy as np

In [4]:
df  = pd.DataFrame(columns = ['album_title','artist','type','date','genre','style','agency',
                             'distributor','album_playtime','number','title','n','title_song_or_not','bugs_score'])
df

,album_title,artist,type,date,genre,style,agency,distributor,album_playtime,number,title,n,title_song_or_not,bugs_score


# 연습

In [5]:
container = dr.find_element_by_id('container')
section = container.find_element_by_css_selector('section[class = "sectionPadding"]')
div  = section.find_element_by_css_selector('div')
lis = div.find_elements_by_css_selector('li')

In [6]:
music_info = {}

## 앨범명

In [7]:
fig = lis[0].find_element_by_css_selector('figcaption')
a = fig.find_element_by_css_selector('a')
music_info['album_title'] = [a.get_attribute('title')]
music_info

{'album_title': ['Miss']}

## 앨범 클릭

In [8]:
#click 앨범 정보 페이지로 이동
a.click()

## 앨범 정보

In [9]:
div = dr.find_element_by_class_name('basicInfo')
tbody = div.find_element_by_css_selector('tbody')
trs = tbody.find_elements_by_css_selector('tr')

In [10]:
#artist는 link 걸려 있는 것도 있고, 아닌 것도 있고
try :
    #link
    a = trs[0].find_element_by_css_selector('a')
    music_info['artist'] = a.text
    
except :
    #not
    td = trs[0].find_element_by_css_selector('td')
    music_info['artist'] = td.text

columns = ['artist','type','date','genre','style','agency','distributor','album_playtime']

for i in range(1,8) :
    td = trs[i].find_element_by_css_selector('td')
    music_info[columns[i-1]] = [td.text]
    
    
music_info

{'album_title': ['Miss'],
 'artist': ['싱글'],
 'type': ['2018.07.29'],
 'date': ['인디, 발라드/댄스/팝'],
 'genre': ['발라드, 인디 팝'],
 'style': ['레이블 숲'],
 'agency': ['NHN벅스'],
 'distributor': ['03:21']}

## 수록곡

In [11]:
section = dr.find_element_by_css_selector('section[class = "sectionPadding contents track"]')
music_info['n'] = [section.find_element_by_css_selector('p').text] #수록곡 개수
music_info

{'album_title': ['Miss'],
 'artist': ['싱글'],
 'type': ['2018.07.29'],
 'date': ['인디, 발라드/댄스/팝'],
 'genre': ['발라드, 인디 팝'],
 'style': ['레이블 숲'],
 'agency': ['NHN벅스'],
 'distributor': ['03:21'],
 'n': ['(1)']}

In [12]:
table = section.find_element_by_css_selector('table')
tbd = table.find_element_by_css_selector('tbody')
trs = tbd.find_elements_by_css_selector('tr')

In [13]:
music_list = []
for tr in trs :
    ps = tr.find_elements_by_css_selector('p')
    
    aa = []
    #서비스를 지원하지 않는 곡들 처리
    try :
        tr.find_element_by_css_selector('input[disabled = "disabled"]')
    except :
        
        for p in ps :
            
            aa.append(p.text)
        
        music_list.append(aa)

In [14]:
d1 = pd.DataFrame(music_list,columns = ['title_song_or_not','title','artist','bugs_score'])
d1

,title_song_or_not,title,artist,bugs_score
0,1\n[타이틀곡],Miss,남소현,선호도 5점


In [15]:
music_info.keys()

dict_keys(['album_title', 'artist', 'type', 'date', 'genre', 'style', 'agency', 'distributor', 'n'])

In [20]:
music_info

{'album_title': ['Miss'],
 'artist': ['싱글'],
 'type': ['2018.07.29'],
 'date': ['인디, 발라드/댄스/팝'],
 'genre': ['발라드, 인디 팝'],
 'style': ['레이블 숲'],
 'agency': ['NHN벅스'],
 'distributor': ['03:21'],
 'n': ['(1)']}

In [16]:
d2 = pd.DataFrame(music_info)
d2 = pd.DataFrame(np.repeat(np.array(d2),len(d1), axis = 0),
                  columns = ['album_title','type','date','genre','style','agency','distributor','album_playtime','n'])
d2

,album_title,type,date,genre,style,agency,distributor,album_playtime,n
0,Miss,싱글,2018.07.29,"인디, 발라드/댄스/팝","발라드, 인디 팝",레이블 숲,NHN벅스,03:21,(1)


In [17]:
d3 = pd.concat([d1,d2], axis = 1)
d3

,title_song_or_not,title,artist,bugs_score,album_title,type,date,genre,style,agency,distributor,album_playtime,n
0,1\n[타이틀곡],Miss,남소현,선호도 5점,Miss,싱글,2018.07.29,"인디, 발라드/댄스/팝","발라드, 인디 팝",레이블 숲,NHN벅스,03:21,(1)


In [18]:
d5 = pd.DataFrame()
d5 = pd.concat([d5,d3], axis = 0)
d5

,title_song_or_not,title,artist,bugs_score,album_title,type,date,genre,style,agency,distributor,album_playtime,n
0,1\n[타이틀곡],Miss,남소현,선호도 5점,Miss,싱글,2018.07.29,"인디, 발라드/댄스/팝","발라드, 인디 팝",레이블 숲,NHN벅스,03:21,(1)


# 한 페이지짜리 함수

In [23]:
def crawling_one_page(pageNo) :
    pageNo = str(pageNo)
    dr.get('https://music.bugs.co.kr/newest/album/kpoppicked?page='+pageNo)
    
    container = dr.find_element_by_id('container')
    section = container.find_element_by_css_selector('section[class = "sectionPadding"]')
    div  = section.find_element_by_css_selector('div')
    lis = div.find_elements_by_css_selector('li')
    
    music_info = {}
    d4 = pd.DataFrame()#final_dataframe
    
    for li in lis :
        
        #album title
        fig = li.find_element_by_css_selector('figcaption')
        a = fig.find_element_by_css_selector('a')
        music_info['album_title'] = [a.get_attribute('title')]
        
        #click 앨범 정보 페이지로 이동
        a.click()
        
        div = dr.find_element_by_class_name('basicInfo')
        tbody = div.find_element_by_css_selector('tbody')
        trs = tbody.find_elements_by_css_selector('tr')
        
        #artist는 link 걸려 있는 것도 있고, 아닌 것도 있고
        try :
            #link
            a = trs[0].find_element_by_css_selector('a')
            music_info['artist'] = a.text

        except :
            #not
            td = trs[0].find_element_by_css_selector('td')
            music_info['artist'] = td.text

        columns = ['type','date','genre','style','agency','distributor','album_playtime']

        for i in range(1,8) :
            td = trs[i].find_element_by_css_selector('td')
            music_info[columns[i-1]] = [td.text]

        
        #수록곡
        section = dr.find_element_by_css_selector('section[class = "sectionPadding contents track"]')
        music_info['n'] = [section.find_element_by_css_selector('p').text] #수록곡 개수
        
        table = section.find_element_by_css_selector('table')
        tbd = table.find_element_by_css_selector('tbody')
        trs = tbd.find_elements_by_css_selector('tr')
        
        music_list = []
        for tr in trs :
            ps = tr.find_elements_by_css_selector('p')

            aa = []
            #서비스를 지원하지 않는 곡들 처리
            try :
                tr.find_element_by_css_selector('input[disabled = "disabled"]')
            except :

                for p in ps :

                    aa.append(p.text)

                music_list.append(aa)
                
        d1 = pd.DataFrame(music_list,columns = ['title_song_or_not','title','artist','bugs_score'])
        print(d1)
        d2 = pd.DataFrame(music_info)
        print(d2)
        d2 = pd.DataFrame(np.repeat(np.array(d2),len(d1), axis = 0),
                  columns = ['album_title','type','date','genre','style','agency','distributor','album_playtime','n'])

        print(d2)
        d3 = pd.concat([d1,d2], axis = 1)
        print(d3)
        d4 = pd.concat([d4,d3], axis = 0)
        d4.to_csv('pageNo_'+pageNo+'.csv')

In [24]:
crawling_one_page(100)

  title_song_or_not title artist bugs_score
0         1\n[타이틀곡]  Miss    남소현     선호도 5점
  album_title artist type        date         genre      style agency  \
0        Miss    남소현   싱글  2018.07.29  인디, 발라드/댄스/팝  발라드, 인디 팝  레이블 숲   

  distributor album_playtime    n  
0       NHN벅스          03:21  (1)  


ValueError: Shape of passed values is (10, 1), indices imply (9, 1)